In [162]:
%reload_ext autoreload
%autoreload 2

import time
from tqdm import tqdm
import numpy as np
import scipy as sp
import torch
import torch.nn.functional as F
from torch import nn
from torchvision.transforms import v2 as transforms

from matplotlib import pyplot as plt

from networkAlignmentAnalysis.models.registry import get_model
from networkAlignmentAnalysis import utils
from networkAlignmentAnalysis import datasets
from networkAlignmentAnalysis import files

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print('using device: ', DEVICE)

using device:  cuda


In [ ]:
# TODO
# 1.1. include additional AlignmentModel methods stored in extra class in base model
# -- the measure_eigenfeature method is baaaaad -- because it doesn't account for convolutional layers

# I should also break apart the measure_eigenfeatures method for better parameter handling
# and also to allow use of components of the method in other contexts

# 2. Make CIFAR class
# 4. Rewrite existing analysis pipelines
# 5. SLURM!!!!


# Analyses:
# - targeted dropout throughout training curves (blue and black in alignmentMNIST.ipynb)
# - compare initial to final alignment...
# - compare initial alignment to delta weight norm...
# - observe alignment of weight norm
# - compare alignment to outgoing weight norm!


# alignmentShaping.ipynb has an adversarial experiment worth looking at